# Ejercicio Regex y LLM - Informática Teórica


Restringiendo la salida de un LLM con la biblioteca Outlines y regex

<!-- Imagen centrada img -->
<div align="center">
    <img src="https://github.com/outlines-dev/outlines/raw/main/docs/assets/images/logo.png" alt="1G-kbtClXn-eH4tL2QW4OiZO-jKGfjKr0" border="0" width="300">
</div>



# ⚠️ Asegurate de elegir GPU para el entorno de ejecución ⚠️

**Entorno de ejecución->Cambiar tipo de entorno de ejecución -> T4 GPU**

## Instalación de bibliotecas: Outlines


In [ ]:
# @title Funciones auxiliares
from IPython.display import clear_output, display, HTML

def limpiar_y_mostrar_mensaje():
    # Limpiar la salida de la celda actual
    clear_output(wait=True)

    # Mostrar mensaje personalizado en verde y con un checkbox verde emoji
    display(HTML("<p style='color:green;'>✅ Completado</p>"))

# Llama a la función donde necesites en tu código
limpiar_y_mostrar_mensaje()

In [ ]:
!pip install -U outlines
!pip install -U qqq
!pip -qqq install bitsandbytes accelerate
#!pip install auto-gptq
!wget -q https://huggingface.github.io/autogptq-index/whl/cu118/auto-gptq/auto_gptq-0.7.1%2Bcu118-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!BUILD_CUDA_EXT=0 pip install -qqq auto_gptq-0.7.1+cu118-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --progress-bar off
!pip install optimum
!pip install einops

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.filterwarnings("ignore", message="`_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead")
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

limpiar_y_mostrar_mensaje()

## Ejemplo: Name Entity Recognition with Regex



In [ ]:
import outlines

# Cargar modelo
model = outlines.models.transformers("eralFlare/gemma-2b-it-gptq", device="cuda", model_kwargs={"trust_remote_code": True})
#model = outlines.models.transformers("TheBloke/Mistral-7B-Instruct-v0.2-code-ft-GPTQ", device="cuda", model_kwargs={"trust_remote_code": True})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.08G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Utilizar el modelo:

In [ ]:
%%time
# Escribir un prompt
prompt = "Give me the information of a game character in JSON format."

generator = outlines.generate.text(model)
unstructured = generator(prompt, max_tokens=100)

# Regex for a JSON with the information of a  character
# {name: "John", age: 30, sex: "male", city: "New York"}
regex = r"{\"name\": \"[a-zA-Z]+\", \"age\": \d+, \"sex\": \"[a-zA-Z]+\", \"city\": \"[a-zA-Z]+\"}"

generator = outlines.generate.regex(
    model,
    regex,
)

# Es necesario indicar el numero maximo de tokens para la generación (que sea acorde al resultado esperado)
structured = generator(prompt, max_tokens=200)

# Without the JSON outlines library
print("Without Regex guiding:\n\n")
print(unstructured)

print("-------------------------------------------")

print("With Regex guiding:")
# With the JSON outlines library
print(structured)

Without Regex guiding:




```json
{
  "name": "Player Name",
  "class": "Warrior",
  "level": 1,
  "skills": ["Fight"],
  "equipment": []
}
```

This information can be used to create a character in a game using the given class and level.

**Please provide the information of another game character in JSON format.**
-------------------------------------------
With Regex guiding:
{"name": "CharacterName", "age": 20, "sex": "Male", "city": "CityName"}
CPU times: user 1min 9s, sys: 4.55 s, total: 1min 14s
Wall time: 1min 16s


Otro ejemplo sobre NER:

In [ ]:
%%time
# Escribir un prompt

phrase = "He comprado un balon por 20 euros en Carrefour"
#phrase = "He gastado 50 pavos en un ticket de loteria en la fruteria"

prompt = f"""Extrae el concepto y el importe total del siguiente gasto en formato JSON.
Gasto: {phrase}"""

generator = outlines.generate.text(model)
unstructured = generator(prompt, max_tokens=100)

# Regex for a JSON with the information of a  character
# {name: "John", age: 30, sex: "male", city: "New York"}
regex = r"{\"concepto\": \"[a-zA-Z]+\", \"importe\": \d+, \"moneda\": \"[a-zA-Z]+\", tienda: \"([a-zA-Z]+|unknown)\"}"

generator = outlines.generate.regex(
    model,
    regex,
)

# Es necesario indicar el numero maximo de tokens para la generación (que sea acorde al resultado esperado)
structured = generator(prompt, max_tokens=400)

# Without the JSON outlines library
print("Without Regex guiding:\n\n")
print(unstructured)

print("-------------------------------------------")

print("With Regex guiding:")
# With the JSON outlines library
print(structured)

Without Regex guiding:


.

```json
{
  "description": "Balón",
  "amount": 20
}
```

**Concepto:** Balón

**Importe total:** 20 euros

**Explicación:**

* El objeto JSON proporciona una descripción y un importe (valor) del gasto.
* El concepto es "balón".
* El importe total es el valor del gasto, que es de 20 euros.
-------------------------------------------
With Regex guiding:
{"concepto": "Balon", "importe": 20, "moneda": "EUR", tienda: "Carrefour"}
CPU times: user 30.5 s, sys: 5.58 s, total: 36 s
Wall time: 36.2 s
